In [10]:
# Step 1: Import Required Libraries
import os
import io
import zipfile
import librosa
import numpy as np
from google.colab import files   # Only if using Google Colab
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [11]:
print("📂 Please upload your audio dataset zip file...")
uploaded = files.upload()  # lets you pick a zip file

for fn, file_content in uploaded.items():
    print(f'User uploaded "{fn}" with length {len(file_content)} bytes')
    if fn.endswith(".zip"):
        with zipfile.ZipFile(io.BytesIO(file_content), 'r') as zip_ref:
            zip_ref.extractall("audio_dataset")   # extract here
print("✅ Dataset extracted to 'audio_dataset/'")

📂 Please upload your audio dataset zip file...


Saving audio_dataset.zip to audio_dataset (1).zip
User uploaded "audio_dataset (1).zip" with length 9206040 bytes
✅ Dataset extracted to 'audio_dataset/'


In [9]:
import zipfile
import io
from google.colab import files

# Upload files and populate the 'uploaded' dictionary
uploaded = files.upload()

# Loop through uploaded files
for fn, file_content in uploaded.items():
    print(f'User uploaded file "{fn}" with length {len(file_content)} bytes')
    if fn.endswith(".zip"):
        with zipfile.ZipFile(io.BytesIO(file_content), 'r') as zip_ref:
            zip_ref.extractall("audio_dataset")

Saving audio_dataset.zip to audio_dataset.zip
User uploaded file "audio_dataset.zip" with length 9206040 bytes


In [12]:
# Step 3: Feature Extraction

def extract_features(file_path):
    """
    Extracts MFCC features from an audio file and returns the mean feature vector.
    """
    audio, sr = librosa.load(file_path, sr=None)  # Load audio
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)  # Extract MFCCs
    return np.mean(mfccs.T, axis=0)  # Average across time frames


In [13]:
# Step 4: Load Dataset
X, y = [], []
data_dir = "audio_dataset/"

print("🔎 Extracting features from audio files...")
for label, folder in enumerate(os.listdir(data_dir)):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):  # only consider subfolders
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                features = extract_features(file_path)
                X.append(features)
                y.append(label)
            except Exception as e:
                print(f"⚠️ Error processing {file_path}: {e}")

🔎 Extracting features from audio files...


In [14]:
X = np.array(X)
y = np.array(y)

print(f"✅ Feature matrix shape: {X.shape}")
print(f"✅ Labels shape: {y.shape}")

✅ Feature matrix shape: (60, 40)
✅ Labels shape: (60,)


In [15]:
# Step 5: Split & Normalize

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(f"🔹 Training samples: {X_train.shape[0]}")
print(f"🔹 Testing samples: {X_test.shape[0]}")

🔹 Training samples: 48
🔹 Testing samples: 12


In [16]:
# Step 6: Build Model

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,521 (170.00 KB)

 Trainable params: 43,521 (170.00 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Step 7: Train Model

history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_split=0.2,
                    batch_size=32)

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 244ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.0

In [18]:
# Step 8: Evaluate Model

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🎯 Test Accuracy: {test_acc:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 1.0000 - loss: 0.0000e+00
🎯 Test Accuracy: 1.0000
